# Clustering Fasttext+W2V (Fast 2 Vec) dengan AP Algorithm

In [ ]:
# mount your drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!gdown https://drive.google.com/uc?id=1BpTkrFqGH4BoDBTcR3tmm9ZZuj6FjlLB
!gdown https://drive.google.com/uc?id=1puCMovgqC32m9AAshbASzGHDEgKvWEvM
!gdown https://drive.google.com/uc?id=1NtebE9VUzA4MvA56EuBSN-OVl7CODHo1

Downloading...
From: https://drive.google.com/uc?id=1BpTkrFqGH4BoDBTcR3tmm9ZZuj6FjlLB
To: /content/requirements.txt
100% 273/273 [00:00<00:00, 1.22MB/s]
Downloading...
From: https://drive.google.com/uc?id=1puCMovgqC32m9AAshbASzGHDEgKvWEvM
To: /content/src.zip
100% 18.1k/18.1k [00:00<00:00, 34.6MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1NtebE9VUzA4MvA56EuBSN-OVl7CODHo1
From (redirected): https://drive.google.com/uc?id=1NtebE9VUzA4MvA56EuBSN-OVl7CODHo1&confirm=t&uuid=fa2b49db-f972-492d-b215-656d64f7f362
To: /content/data.zip
100% 76.5M/76.5M [00:00<00:00, 86.1MB/s]


In [ ]:
# extract data and source code (src)
!unzip /content/src.zip -d /content/
!unzip /content/data.zip -d /content/

Archive:  /content/src.zip
 extracting: /content/src/__init__.py  
   creating: /content/src/__pycache__/
  inflating: /content/src/__pycache__/__init__.cpython-37.pyc  
  inflating: /content/src/__pycache__/static_method.cpython-37.pyc  
  inflating: /content/src/__pycache__/topic_detection.cpython-37.pyc  
  inflating: /content/src/__pycache__/topic_semantic.cpython-37.pyc  
  inflating: /content/src/static_method.py  
  inflating: /content/src/topic_detection.py  
  inflating: /content/src/topic_semantic.py  
Archive:  /content/data.zip
  inflating: /content/data/centroid_fasttext_menggunakan_stopwords.csv  
  inflating: /content/data/centroid_fasttext_w2v_menggunakan_stopwords.csv  
  inflating: /content/data/centroid_w2v_menggunakan_stopwords.csv  
  inflating: /content/data/data_jurnal_clean.csv  
  inflating: /content/data/data_jurnal_fix.csv  
  inflating: /content/data/hasil_cluster_fasttext_menggunakan_stopwords.csv  
  inflating: /content/data/hasil_cluster_w2v_menggunakan_s

In [ ]:
# create env and install package
!pip3 install virtualenv
!virtualenv /content/disertasi
!source /content/disertasi; pip install --upgrade pip
!source /content/disertasi; pip install ipykernel
!source /content/disertasi; pip install -r /content/requirements.txt

%cd /content/
%ls

# !pip install networkx
!pip install numpy==1.25.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 21.9 MB/s eta 0:00:00
created virtual environment CPython3.10.12.final.0-64 in 1778ms
  creator CPython3Posix(dest=/content/disertasi, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==24.2, setuptools==75.1.0, wheel==0.44.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
/bin/bash: line 1: source: /content/disertasi: is a directory
  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
/bin/bash: line 1: source: /content/disertasi: is a d

## Stop here and restart your session

In [ ]:
# import Library
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from warnings import filterwarnings
filterwarnings('ignore')
from jcopml.utils import save_model, load_model
from string import punctuation

from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec, FastText
import gensim
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.manifold import TSNE
from sklearn.cluster import AffinityPropagation, MiniBatchKMeans, KMeans
from umap import UMAP
from sklearn.decomposition import PCA
import plotly.express as px

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [ ]:
def reduce_dimension(word_vector_model, reduction_technique='umap', n_components=2, random_state=42):

    if reduction_technique == 'tsne':
        results = TSNE(n_components=n_components, random_state=random_state).fit_transform(word_vector_model.vectors)
    elif reduction_technique == 'umap':
        results = UMAP(n_components=n_components, random_state=random_state).fit_transform(word_vector_model.vectors)
    elif reduction_technique == 'pca':
        results = PCA(n_components=n_components, random_state=random_state).fit_transform(word_vector_model.vectors)
    else:
        raise Exception('reduction_technique only support {tsne, umap, pca}')

    df = pd.DataFrame(results)
    df.insert(0, 'word', word_vector_model.index_to_key)

    return df

## Load Model

In [ ]:
from jcopml.utils import load_model

In [ ]:
# load model with stopwords
ft_with_sw = load_model('/content/drive/MyDrive/Upload_Github/model/fasttext_modif/model_fasttext_abstract_with_sw_modif.pkl')

## Reduce Dimension with TSNE

In [ ]:
# # with stopwords
# df_tsne_with_sw = reduce_dimension(ft_with_sw, reduction_technique='tsne')

In [ ]:
# load data tsne
df_tsne_with_sw = pd.read_csv('/content/drive/MyDrive/Upload_Github/extracted data/df_tsne_with_sw_ft+w2v.csv')

In [ ]:
df_tsne_with_sw.head()

,word,0,1
0,student,41.954740,4.170799
1,system,-16.803204,38.174324
2,analysis,7.463954,68.511696
3,model,-16.166843,55.346752
4,value,59.651585,0.355062


In [ ]:
df_tsne_with_sw.dropna(inplace=True)

## Clustering With Affinity Propagation

### Clustering Fast 2 Vec with Stopwords (SW)

In [ ]:
def clustering_word_vector(df, columns, cluster_method='ap'):

    if cluster_method == 'ap':
        start = time.time()
        ap = AffinityPropagation(damping=0.9, max_iter=1000, random_state=42)
        cluster = ap.fit_predict(df[columns])
        centroid = ap.cluster_centers_
        stop =  time.time()
        print(f"Training time: {stop - start}s")

    return cluster, centroid

In [ ]:
# cluster_with_sw, centroid_with_sw = clustering_word_vector(df_tsne_with_sw, columns=[0, 1])

Training time: 1431.6693024635315s


In [ ]:
# df_results_with_sw = pd.DataFrame(df_tsne_with_sw).copy()
# df_results_with_sw['cluster'] = cluster_with_sw
# df_results_with_sw.head()

,word,0,1,cluster
0,student,41.954739,4.170799,134
1,system,-16.803204,38.174324,50
2,analysis,7.463954,68.511696,26
3,model,-16.166843,55.346752,43
4,value,59.651585,0.355062,8


In [ ]:
# dfC_with_sw = pd.DataFrame(centroid_with_sw, columns=pd.DataFrame(df_tsne_with_sw).columns[1:])
# dfC_with_sw['cluster'] = dfC_with_sw.index
# dfC_with_sw.head()

,0,1,cluster
0,56.868450,-22.510088,0
1,61.494003,21.216877,1
2,61.982437,-11.424911,2
3,-4.870079,64.081055,3
4,52.861702,45.433353,4


In [ ]:
# df_results_with_sw.to_csv('data/hasil_cluster_fasttext_w2v_menggunakan_stopwords.csv', index=False)
# dfC_with_sw.to_csv('data/centroid_fasttext_w2v_menggunakan_stopwords.csv', index=False)

##### Visualization Cluster With Plotly

In [ ]:
def visualize_cluster(df, x, y, cluster_columns, title, color_continuous_scale=None, height=800):
    fig = px.scatter(df, x=x, y=y, color=cluster_columns, color_continuous_scale=color_continuous_scale)
    fig.update_traces(textposition='top center')
    fig.update_layout(
        height=height,
        title_text=title
    )
    fig.show()

In [ ]:
# load dataset results clustering
df_results_with_sw = pd.read_csv('/content/data/x_hasil_cluster_fasttext_w2v_menggunakan_stopwords.csv')
dfC_with_sw = pd.read_csv('/content/data/centroid_fasttext_w2v_menggunakan_stopwords.csv')

In [ ]:
top_20 = df_results_with_sw.cluster.value_counts().head(20).index.tolist()
df_results_with_sw_top20 = df_results_with_sw[df_results_with_sw.cluster.isin(top_20)]

In [ ]:
colors = ['red','green','blue','pink','chocolate','yellow','turquoise','navy','purple','mintcream',
         'orange','olive','palevioletred','gold','skyblue','orangered','seagreen','darkslategrey','darkcyan','mediumpurple']

In [ ]:
len(colors)

20

In [ ]:
visualize_cluster(df_results_with_sw, x='0', y='1', cluster_columns='cluster',
                  title='Cluster Visualization With Stopwords')

In [ ]:
visualize_cluster(df_results_with_sw_top20, x='0', y='1', cluster_columns='cluster',
                  title='Top 20 Cluster Visualization With Stopwords', color_continuous_scale=colors)